In [50]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
from keras.layers import Dense,Input,LSTM,Bidirectional,Activation,Conv1D,GRU
from keras.callbacks import Callback
from keras.layers import Dropout,Embedding,GlobalMaxPooling1D, MaxPooling1D, Add, Flatten
from keras.preprocessing import text, sequence
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.models import Model
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
#print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [51]:
EMBEDDING_FILE = './toxicdata17/glove.840B.300d.txt'
train = pd.read_csv('./toxicdata17/train.csv')
test = pd.read_csv('./toxicdata17/test.csv')
print(test.shape)

(153164, 2)


In [52]:
train["comment_text"].fillna("fillna")
test["comment_text"].fillna("fillna")
X_train = train["comment_text"].str.lower()
y_train = train[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values

X_test = test["comment_text"].str.lower()

In [53]:
max_features=100000
maxlen=150
embed_size=300

In [54]:
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: {:d} - score: {:.6f}".format(epoch+1, score))

In [55]:
tok=text.Tokenizer(num_words=max_features,lower=True)
tok.fit_on_texts(list(X_train)+list(X_test))
X_train=tok.texts_to_sequences(X_train)
X_test=tok.texts_to_sequences(X_test)
x_train=sequence.pad_sequences(X_train,maxlen=maxlen)
x_test=sequence.pad_sequences(X_test,maxlen=maxlen)
#print(x_test)

In [56]:
embeddings_index = {}
with open(EMBEDDING_FILE,encoding='utf8') as f:
    for line in f:
        values = line.rstrip().rsplit(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs


In [57]:
word_index = tok.word_index
#prepare embedding matrix
num_words = min(max_features, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, embed_size))
for word, i in word_index.items():
    if i >= max_features:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [58]:
sequence_input = Input(shape=(maxlen, ))
x = Embedding(max_features, embed_size, weights=[embedding_matrix],trainable = False)(sequence_input)
x = SpatialDropout1D(0.2)(x)
x = Bidirectional(GRU(128, return_sequences=True,dropout=0.1,recurrent_dropout=0.1))(x)
x = Conv1D(64, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")(x)
avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
x = concatenate([avg_pool, max_pool]) 
# x = Dense(128, activation='relu')(x)
# x = Dropout(0.1)(x)
preds = Dense(6, activation="sigmoid")(x)
model = Model(sequence_input, preds)
model.compile(loss='binary_crossentropy',optimizer=Adam(lr=1e-3),metrics=['accuracy'])

In [59]:
batch_size = 128
epochs = 2
X_tra, X_val, y_tra, y_val = train_test_split(x_train, y_train, train_size=0.9, random_state=233)

In [60]:
# filepath="../input/best-model/best.hdf5"
filepath='./toxicdata17/best.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=5)
ra_val = RocAucEvaluation(validation_data=(X_val, y_val), interval = 1)
callbacks_list = [ra_val,checkpoint, early]

In [61]:
model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val),callbacks = callbacks_list,verbose=1)
#Loading model weights
#model.load_weights(filepath)
#print('Predicting....')


Train on 143613 samples, validate on 15958 samples
Epoch 1/2
143613/143613 [==============================] - 427s 3ms/step - loss: 4.8190 - accuracy: 0.9781 - val_loss: 0.0589 - val_accuracy: 0.9797

 ROC-AUC - epoch: 1 - score: 0.943486
Epoch 2/2


/home/amiangshu/.local/lib/python3.6/site-packages/keras/callbacks/callbacks.py:707: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)
/home/amiangshu/.local/lib/python3.6/site-packages/keras/callbacks/callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_acc` which is not available. Available metrics are: val_loss,val_accuracy,loss,accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


143613/143613 [==============================] - 424s 3ms/step - loss: 9.7206 - accuracy: 0.9741 - val_loss: 0.1047 - val_accuracy: 0.9694

 ROC-AUC - epoch: 2 - score: 0.741748


/home/amiangshu/.local/lib/python3.6/site-packages/keras/callbacks/callbacks.py:707: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)
/home/amiangshu/.local/lib/python3.6/site-packages/keras/callbacks/callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_acc` which is not available. Available metrics are: val_loss,val_accuracy,loss,accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


In [62]:
y_pred = model.predict(x_test,batch_size=1024,verbose=1)





153164/153164 [==============================] - 145s 945us/step


In [63]:
print(x_test.shape)
print(y_pred.shape)

(153164, 150)
(153164, 6)


In [32]:
print(y_pred)

[[9.91111398e-01 3.71329784e-02 9.74681854e-01 3.12766433e-03
  8.04336071e-01 4.48227823e-02]
 [8.43595743e-01 1.05930865e-02 2.72698760e-01 4.75543737e-03
  3.57436419e-01 1.09182894e-02]
 [3.03560317e-01 1.98203325e-03 2.08299756e-02 4.85420227e-04
  1.09441549e-01 9.41458344e-03]
 ...
 [2.90298939e-01 2.08209753e-02 1.77500099e-01 2.44193673e-02
  3.11787546e-01 2.43442953e-02]
 [3.14978957e-02 7.54609704e-03 1.07543766e-02 2.44617462e-04
  3.21825743e-02 6.43074512e-03]
 [1.85311735e-02 4.88263369e-03 5.25388122e-03 1.07645988e-04
  6.66850805e-03 3.13264132e-03]]


In [65]:
submission = pd.read_csv('./toxicdata17/sample_submission.csv')
print(submission.shape)
submission[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]] = y_pred
#submission[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]] = y_pred

(153164, 7)


In [67]:
submission.to_csv('./toxicdata17/submission.csv', index=False)